In [52]:
# get info of colab GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 26 01:45:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |   2900MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [53]:
# Check the GPU device is working
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [54]:
# Access to the google drive where the data is stored
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [55]:
# Libraries
from tensorflow import keras
from keras.models import Sequential 
import cv2
import os
from pathlib import Path
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from skimage import io
import glob
import random
from sklearn.model_selection import train_test_split



In [56]:
from operator import concat
#Define Paths
data_train_path = Path('/content/drive/My Drive/data/FaceData/train')         # Define path for data
data_test_path = Path('/content/drive/My Drive/data/FaceData/test')         # Define path for data


def path_produce(premiere_path,emotion_path):
  y = premiere_path/emotion_path
  return y
emotion = ["angry", "disgusted", "fearful", "happy","neutral", "sad", "surprised"]
emotion_number = ["0", "1", "2", "3","4", "5", "6"]


def get_images(images_path): 
  return images_path.glob('*.png')




def get_path_give_images(path,emotion,emotion_number):
  all_data = pd.DataFrame(columns=['image' , 'emotionclass'],index=None)
  j=0;
  for i in emotion:
    EmPath = path_produce(path, i)
    array_images  = get_images(EmPath) 
    data = pd.DataFrame(array_images, columns=['image'],index=None)
    data['emotionclass'] = emotion_number[j]
    j+=1
    d=[all_data,data]
    all_data = pd.concat(d)
  return all_data

# Get data from paths
all_train_data = get_path_give_images(data_train_path,emotion,emotion_number)
all_test_data = get_path_give_images(data_test_path,emotion,emotion_number)

# Shuffle data
train_data = all_train_data.sample(frac=1.).reset_index(drop=True)
test_data = all_test_data.sample(frac=1.).reset_index(drop=True)

X = train_data.image;
y = train_data.emotionclass;


X_val, X_train, y_val , y_train = train_test_split(X, y, test_size=0.9, random_state=1)
df1 = {'image': X_train, 'emotionclass': y_train}
train_data = pd.DataFrame(df1,index=None)

df2 = {'image': X_val, 'emotionclass': y_val}
validation_data = pd.DataFrame(df2,index=None)



In [57]:
all_train_data.groupby('emotionclass').emotionclass.count()

emotionclass
0    3995
1     436
2    4097
3    7215
4    4965
5    4830
6    3171
Name: emotionclass, dtype: int64

In [58]:
# Parameters
IMG_SIZE = 256  # Image size
BATCH_SIZE = 256

# Preprocessing object
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

# Turn values to strings as a requirement for flow_from_dataframe
train_data['image'] = train_data['image'].astype("string")
validation_data['image'] = validation_data['image'].astype("string")
test_data['image'] = test_data['image'].astype("string")

# Create train generator
train_gen = idg.flow_from_dataframe(dataframe= train_data,
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )

# Create test generator
validation_gen = idg.flow_from_dataframe(dataframe= validation_data, 
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )
# Create test generator
test_gen = idg.flow_from_dataframe(dataframe= test_data, 
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )

Found 25839 validated image filenames belonging to 7 classes.
Found 2870 validated image filenames belonging to 7 classes.
Found 7178 validated image filenames belonging to 7 classes.


In [83]:

#Define CNN model

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
import tensorflow as tf


emotion_model = Sequential()

# Block1
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# emotion_model.add(Dropout(0.2))

# Block2
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# emotion_model.add(Dropout(0.2))


# Block4
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
# emotion_model.add(BatchNormalization())
# emotion_model.add(Dropout(0.2))
emotion_model.add(Dense(7, activation='softmax'))

In [84]:
emotion_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6), metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [85]:
#Train Model
# from keras.models import load_model

# TRAINING = True
NUM_EPOCHS = 30
# model_path = '/content/drive/MyDrive/Project_Colab/Pneumonia_detection/my_model.h5'

# if TRAINING:
# emotion_model_info = emotion_model.fit_generator(train_gen, 
#                                         epochs=NUM_EPOCHS, 
#                                         steps_per_epoch=None,
#                                         validation_data=validation_gen,
#                                         validation_steps=len(validation_gen))

emotion_model_info = emotion_model.fit(
        train_gen,
        steps_per_epoch=len(train_gen)//2,
        epochs=NUM_EPOCHS,
        validation_data=validation_gen,
        validation_steps=len(validation_gen)//2)
   
# else:

  # returns a compiled model
  # identical to the previous one
  # model = load_model(model_path, compile=True)

Epoch 1/30
50/50 [==============================] - 17s 319ms/step - loss: 1.8648 - accuracy: 0.2632 - val_loss: 1.9040 - val_accuracy: 0.2389
Epoch 2/30
50/50 [==============================] - 16s 311ms/step - loss: 1.5967 - accuracy: 0.3770 - val_loss: 1.8614 - val_accuracy: 0.3607
Epoch 3/30
50/50 [==============================] - 15s 305ms/step - loss: 1.4422 - accuracy: 0.4493 - val_loss: 1.8076 - val_accuracy: 0.3698
Epoch 4/30
50/50 [==============================] - 15s 308ms/step - loss: 1.3454 - accuracy: 0.4879 - val_loss: 1.7946 - val_accuracy: 0.3750
Epoch 5/30
50/50 [==============================] - 15s 308ms/step - loss: 1.2774 - accuracy: 0.5142 - val_loss: 1.7576 - val_accuracy: 0.4193
Epoch 6/30
50/50 [==============================] - 16s 318ms/step - loss: 1.2100 - accuracy: 0.5433 - val_loss: 1.6619 - val_accuracy: 0.4674
Epoch 7/30
50/50 [==============================] - 16s 318ms/step - loss: 1.1629 - accuracy: 0.5627 - val_loss: 1.5926 - val_accuracy: 0.4824

In [126]:
# predict test data
num_test_batchs = len(iter(test_gen))

pred_list = []
label_list = []

for i in range(num_test_batchs):
  image, label = test_gen[i]
  out_model = emotion_model.predict(
    image, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1
    )
  pred = tf.math.greater(out_model, 0.25, name=None)
  pred_ = tf.dtypes.cast(pred, tf.int32)
  pred_list.append(pred_)
  label_list.append(label)

In [127]:
out_model

array([[7.85024667e-07, 9.41863580e-16, 1.37478366e-07, 4.10564971e-05,
        9.99934912e-01, 2.31178619e-05, 1.79220960e-09],
       [9.99059618e-01, 2.35499738e-05, 8.56914907e-04, 5.72191175e-06,
        9.63712955e-06, 3.26627378e-05, 1.18548105e-05],
       [1.08728716e-04, 5.46257007e-12, 2.52455589e-04, 6.57899245e-13,
        3.72752245e-03, 9.95911360e-01, 1.63603930e-09],
       [4.55349655e-04, 1.20028830e-07, 9.66539383e-01, 1.78569257e-02,
        6.84433462e-06, 1.61701319e-04, 1.49796642e-02],
       [2.34723699e-13, 4.03686986e-24, 7.17456317e-07, 7.46288197e-06,
        1.85606463e-04, 9.99806225e-01, 1.71379647e-10],
       [7.39901404e-11, 8.31267672e-13, 8.83622168e-12, 5.33264415e-15,
        8.56255267e-18, 7.36446928e-13, 1.00000000e+00],
       [2.27521081e-03, 1.79301537e-06, 1.48523524e-01, 1.06665688e-02,
        1.38896480e-02, 8.22471142e-01, 2.17206730e-03],
       [4.42705071e-03, 2.94305980e-01, 1.18316861e-03, 3.38081081e-05,
        1.10743269e-02, 6

In [128]:
# Find y_test
t = pd.DataFrame(np.array((label_list)))
y_test = pd.DataFrame([])
tt= np.asarray(t).flatten()
for i in range(num_test_batchs):
  ttt = np.reshape(tt[i], (-1,7))
  y_test = y_test.append(pd.DataFrame(ttt))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [129]:
#Find y_prediction

pred_ = pd.DataFrame(np.array((pred_list)))
y_predict = pd.DataFrame([])
pp= np.asarray(pred_).flatten()
for i in range(num_test_batchs):
  ppp = np.reshape(pp[i], (-1,7))
  y_predict = y_predict.append(pd.DataFrame(ppp))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [130]:
# Evaluate Result
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score,precision_score,recall_score
# print("confusion_matrix", confusion_matrix(y_test,y_predict))
from sklearn.metrics import multilabel_confusion_matrix

print(multilabel_confusion_matrix(y_test, y_predict))
print("Accuracy", accuracy_score(y_test, y_predict))
print("F1_Score", f1_score(y_test,y_predict, average = 'macro'))
print("Precision:",precision_score(y_test, y_predict,average='macro'))
print("recall",recall_score(y_test, y_predict,average='macro'))

[[[5789  431]
  [ 521  437]]

 [[6994   73]
  [  52   59]]

 [[5410  744]
  [ 575  449]]

 [[4900  504]
  [ 327 1447]]

 [[5098  847]
  [ 522  711]]

 [[4801 1130]
  [ 540  707]]

 [[6112  235]
  [ 231  600]]]
Accuracy 0.5104485929228197
F1_Score 0.5477830788428486
Precision: 0.5183203962117817
recall 0.5867803229250697
